In [1]:
# Packages that allow us to get information about objects:
import asdf
import copy
import os
import shutil
import glob
# Numpy library:
import numpy as np
# For downloading data
import requests
# Astropy tools:
from astropy.io import fits
from astropy.utils.data import download_file
from astropy.visualization import ImageNormalize, ManualInterval, LogStretch
#Set up matplotlib for plotting
import matplotlib.pyplot as plt
import matplotlib as mpl
# parallelisation
#from joblib import Parallel, delayed #besoin d'installer joblib

#solution tempo du notebook MRS
import multiprocessing
multiprocessing.set_start_method('fork')
from multiprocessing import Pool
# Set the maximum number of processes to spawn based on available cores
usage = 'none' # Either 'none' (single thread), 'quarter', 'half', or 'all' available cores
maxp = 1
print('We will use '+str(maxp)+' CPUs in this run')

#Import JWST pipeline-related modules
import jwst
from jwst.pipeline import Detector1Pipeline
from jwst.pipeline import Image2Pipeline
from jwst.pipeline import Coron3Pipeline
from jwst.associations import asn_from_list as afl # Tools for creating association files
from jwst.associations.lib.rules_level2_base import DMSLevel2bBase # Definition of a Lvl2 association file
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base # Definition of a Lvl3 association file
from jwst.associations.lib.member import Member
from jwst.associations.asn_from_list import asn_from_list
from tqdm.notebook import tqdm as tqdm
#Check which version of the pipeline we are running:
print(jwst.__version__)



We will use 1 CPUs in this run
1.4.3


In [2]:
path_code   = '/home/cperrot/Work/code/MIRISIM/corono/'
path_raw    = '/home/cperrot/Work/code/MIRISIM/corono/20220328_162102_mirisim/det_images/'
path_stage1 = '/home/cperrot/Work/code/MIRISIM/corono/corono_S1/'
path_stage2 = '/home/cperrot/Work/code/MIRISIM/corono/corono_S2/'
path_stage3 = '/home/cperrot/Work/code/MIRISIM/corono/corono_S3/'

if not os.path.exists(path_stage1):
    os.makedirs(path_stage1)
if not os.path.exists(path_stage2):
    os.makedirs(path_stage2)
if not os.path.exists(path_stage3):
    os.makedirs(path_stage3)

FUNCTION

In [3]:
def rundet1(filenames):
    det1 = Detector1Pipeline() # Instantiate the pipeline
    det1.output_dir = path_stage1 # Specify where the output should go
    det1.refpix.skip = True # Skip the reference pixel subtraction (as it doesn't interact well with simulated data)
    det1.rscd.skip = True
    det1.reset.skip = True
    det1.save_results = True # Save the final resulting _rate.fits files
    det1(filenames) # Run the pipeline on an input list of files

In [4]:
# Define a function that can oversee the multiprocessing
# We'll just pass this a function name and a set of inputs, and it'll figure out the
# multithreading accordingly.
def runmany(step, filenames):
    if __name__ == '__main__':
        p = Pool(maxp)
        res = p.map(step, filenames)
        p.close()
        p.join()

In [5]:
def writel2asn(files, asn_name, prodname,**kwargs):
    
    asn = asn_from_list(files, rule=DMSLevel2bBase, product_name=prodname)
    asn.data['products'] = None
    asn['asn_pool'] = asn_name + '_pool'
    asn['asn_type'] = 'image2'
    for n, sci in enumerate(files):
            asn.new_product('{}_exposure{}'.format(asn_name, str(n+1)))
            sci_member = Member({'expname': sci, 'exptype': 'science'})    
            new_members = asn.current_product['members']
            new_members.append(sci_member)
    # print the association and save to file
    name, ser = asn.dump()
    print(ser)
    asn_file2 = path_stage2+asn_name+'.json'
    with open(asn_file2, 'w') as f:
        f.write(ser)

In [6]:
sstring = path_raw+'det*.fits'
raw_files = sorted(glob.glob(sstring))

In [9]:
# RUN STAGE 1
print("Run stage 1 of the pipeline")
print(f'Found {len(raw_files)} input files to process')

Run stage 1 of the pipeline
Found 2 input files to process


In [8]:
runmany(rundet1, raw_files)

2022-04-25 17:49:41,568 - stpipe.Detector1Pipeline - INFO - Detector1Pipeline instance created.
2022-04-25 17:49:41,571 - stpipe.Detector1Pipeline.group_scale - INFO - GroupScaleStep instance created.
2022-04-25 17:49:41,573 - stpipe.Detector1Pipeline.dq_init - INFO - DQInitStep instance created.
2022-04-25 17:49:41,575 - stpipe.Detector1Pipeline.saturation - INFO - SaturationStep instance created.
2022-04-25 17:49:41,578 - stpipe.Detector1Pipeline.ipc - INFO - IPCStep instance created.
2022-04-25 17:49:41,581 - stpipe.Detector1Pipeline.superbias - INFO - SuperBiasStep instance created.
2022-04-25 17:49:41,583 - stpipe.Detector1Pipeline.refpix - INFO - RefPixStep instance created.
2022-04-25 17:49:41,585 - stpipe.Detector1Pipeline.rscd - INFO - RscdStep instance created.
2022-04-25 17:49:41,587 - stpipe.Detector1Pipeline.firstframe - INFO - FirstFrameStep instance created.
2022-04-25 17:49:41,588 - stpipe.Detector1Pipeline.lastframe - INFO - LastFrameStep instance created.
2022-04-25 1

2022-04-25 17:49:43,951 - stpipe.Detector1Pipeline - INFO - Prefetch for PERSAT reference file is 'N/A'.
2022-04-25 17:49:43,952 - stpipe.Detector1Pipeline - INFO - Prefetch for READNOISE reference file is '/home/cperrot/crds_cache/references/jwst/miri/jwst_miri_readnoise_0057.fits'.
2022-04-25 17:49:43,953 - stpipe.Detector1Pipeline - INFO - Prefetch for SATURATION reference file is '/home/cperrot/crds_cache/references/jwst/miri/jwst_miri_saturation_0026.fits'.
2022-04-25 17:49:43,954 - stpipe.Detector1Pipeline - INFO - Prefetch for SUPERBIAS reference file is 'N/A'.
2022-04-25 17:49:43,954 - stpipe.Detector1Pipeline - INFO - Prefetch for TRAPDENSITY reference file is 'N/A'.
2022-04-25 17:49:43,955 - stpipe.Detector1Pipeline - INFO - Prefetch for TRAPPARS reference file is 'N/A'.
2022-04-25 17:49:43,956 - stpipe.Detector1Pipeline - INFO - Starting calwebb_detector1 ...
2022-04-25 17:49:44,139 - stpipe.Detector1Pipeline.group_scale - INFO - Step group_scale running with args (<RampMode

2022-04-25 17:49:46,438 - stpipe.Detector1Pipeline.rscd - INFO - Step skipped.
2022-04-25 17:49:46,441 - stpipe.Detector1Pipeline.rscd - INFO - Step rscd done
2022-04-25 17:49:46,519 - stpipe.Detector1Pipeline.dark_current - INFO - Step dark_current running with args (<RampModel(3, 5, 1024, 1032) from det_image_seq1_MIRIMAGE_F1000Wexp1.fits>,).
2022-04-25 17:49:46,522 - stpipe.Detector1Pipeline.dark_current - INFO - Step dark_current parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': '/home/cperrot/Work/code/MIRISIM/corono/corono_S1/', 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '/home/cperrot/Work/code/MIRISIM/corono/20220328_162102_mirisim/det_images', 'dark_output': None}
2022-04-25 17:49:46,550 - stpipe.Detector1Pipeline.dark_current - INFO - Using DARK reference file /home/cperrot/crds_cache/references/jwst/miri/jwst_mir

2022-04-25 17:49:54,150 - stpipe.Detector1Pipeline.gain_scale - INFO - GAINFACT not found in gain reference file
2022-04-25 17:49:54,151 - stpipe.Detector1Pipeline.gain_scale - INFO - Step will be skipped
2022-04-25 17:49:54,157 - stpipe.Detector1Pipeline.gain_scale - INFO - Step gain_scale done
2022-04-25 17:49:54,227 - stpipe.Detector1Pipeline.gain_scale - INFO - Step gain_scale running with args (<CubeModel(3, 1024, 1032) from det_image_seq1_MIRIMAGE_F1000Wexp1.fits>,).
2022-04-25 17:49:54,229 - stpipe.Detector1Pipeline.gain_scale - INFO - Step gain_scale parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': 'gain_scaleints', 'search_output_file': True, 'input_dir': '/home/cperrot/Work/code/MIRISIM/corono/20220328_162102_mirisim/det_images'}
2022-04-25 17:49:54,278 - stpipe.Detector1Pipeline.gain_scale - INFO - GAINFACT no

2022-04-25 17:49:54,701 - stpipe.Detector1Pipeline - INFO - Prefetching reference files for dataset: 'det_image_seq1_MIRIMAGE_F1000Wexp2.fits' reftypes = ['dark', 'gain', 'ipc', 'linearity', 'mask', 'persat', 'readnoise', 'saturation', 'superbias', 'trapdensity', 'trappars']
2022-04-25 17:49:54,705 - stpipe.Detector1Pipeline - INFO - Prefetch for DARK reference file is '/home/cperrot/crds_cache/references/jwst/miri/jwst_miri_dark_0058.fits'.
2022-04-25 17:49:54,706 - stpipe.Detector1Pipeline - INFO - Prefetch for GAIN reference file is '/home/cperrot/crds_cache/references/jwst/miri/jwst_miri_gain_0008.fits'.
2022-04-25 17:49:54,707 - stpipe.Detector1Pipeline - INFO - Prefetch for IPC reference file is '/home/cperrot/crds_cache/references/jwst/miri/jwst_miri_ipc_0009.fits'.
2022-04-25 17:49:54,707 - stpipe.Detector1Pipeline - INFO - Prefetch for LINEARITY reference file is '/home/cperrot/crds_cache/references/jwst/miri/jwst_miri_linearity_0024.fits'.
2022-04-25 17:49:54,708 - stpipe.Det

2022-04-25 17:49:56,554 - stpipe.Detector1Pipeline.linearity - INFO - Using Linearity reference file /home/cperrot/crds_cache/references/jwst/miri/jwst_miri_linearity_0024.fits
2022-04-25 17:49:56,882 - stpipe.Detector1Pipeline.linearity - INFO - Step linearity done
2022-04-25 17:49:56,960 - stpipe.Detector1Pipeline.rscd - INFO - Step rscd running with args (<RampModel(3, 5, 1024, 1032) from det_image_seq1_MIRIMAGE_F1000Wexp2.fits>,).
2022-04-25 17:49:56,961 - stpipe.Detector1Pipeline.rscd - INFO - Step rscd parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': True, 'suffix': None, 'search_output_file': True, 'input_dir': '/home/cperrot/Work/code/MIRISIM/corono/20220328_162102_mirisim/det_images', 'type': 'baseline'}
2022-04-25 17:49:56,962 - stpipe.Detector1Pipeline.rscd - INFO - Step skipped.
2022-04-25 17:49:56,966 - stpipe.Detector1Pipel

2022-04-25 17:50:04,823 - stpipe.Detector1Pipeline.ramp_fit - INFO - Number of integrations: 3
2022-04-25 17:50:04,904 - stpipe.Detector1Pipeline.ramp_fit - INFO - Step ramp_fit done
2022-04-25 17:50:04,982 - stpipe.Detector1Pipeline.gain_scale - INFO - Step gain_scale running with args (<ImageModel(1024, 1032) from det_image_seq1_MIRIMAGE_F1000Wexp2.fits>,).
2022-04-25 17:50:04,984 - stpipe.Detector1Pipeline.gain_scale - INFO - Step gain_scale parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': 'gain_scale', 'search_output_file': True, 'input_dir': '/home/cperrot/Work/code/MIRISIM/corono/20220328_162102_mirisim/det_images'}
2022-04-25 17:50:05,034 - stpipe.Detector1Pipeline.gain_scale - INFO - GAINFACT not found in gain reference file
2022-04-25 17:50:05,035 - stpipe.Detector1Pipeline.gain_scale - INFO - Step will be skipp

In [10]:
sstring = path_stage1+'det*rate.fits'
stage1_files = sorted(glob.glob(sstring))
print('Found ' + str(len(stage1_files)) + ' input files to process')

Found 2 input files to process


In [11]:
writel2asn(stage1_files, 'lvl2-asn','Level2')

{
    "asn_type": "image2",
    "asn_rule": "DMSLevel2bBase",
    "version_id": null,
    "code_version": "1.4.3",
    "degraded_status": "No known degraded exposures in association.",
    "program": "noprogram",
    "constraints": "No constraints",
    "asn_id": "a3001",
    "asn_pool": "lvl2-asn_pool",
    "products": [
        {
            "name": "lvl2-asn_exposure1",
            "members": [
                {
                    "expname": "/home/cperrot/Work/code/MIRISIM/corono/corono_S1/det_image_seq1_MIRIMAGE_F1000Wexp1_rate.fits",
                    "exptype": "science"
                }
            ]
        },
        {
            "name": "lvl2-asn_exposure2",
            "members": [
                {
                    "expname": "/home/cperrot/Work/code/MIRISIM/corono/corono_S1/det_image_seq1_MIRIMAGE_F1000Wexp2_rate.fits",
                    "exptype": "science"
                }
            ]
        }
    ]
}


In [12]:
# RUN STAGE 2
print("Run stage 2 of the pipeline")

Run stage 2 of the pipeline


Function du stage 2

In [17]:
def runimage2(filename):
    image2 = Image2Pipeline()
    image2.output_dir = path_stage2
    image2.background.skip = False #
    image2.assign_wcs.skip = False #  Derives the world coordinate solution- never skip this! 
    image2.flat_field.skip = False #  Applies a pixel flatfield (currently all unity)
    image2.photom.skip = False # Applies photometric calibration flat
    image2.resample.skip = False # Resampling is only performed for exposure types “MIR_IMAGE”, “NRC_IMAGE”, and “NIS_IMAGE”.

    
    image2.save_results = True
    image2(filename)

In [18]:
def writel3asn(files,asnfile,prodname,**kwargs):
    # Define the basic association of science files
    asn = afl.asn_from_list(files,rule=DMS_Level3_Base,product_name=prodname)
    # Add any background files to the association
    if ('bg' in kwargs):
        for bgfile in kwargs['bg']:
            asn['products'][0]['members'].append({'expname': bgfile, 'exptype':'background'})
    # Write the association to a json file
    _, serialized = asn.dump()
    with open(path_stage3+asnfile, 'w') as outfile:
        outfile.write(serialized)

In [ ]:
runmany(runimage2, stage1_files)

2022-04-25 17:58:25,477 - stpipe.Image2Pipeline - INFO - Image2Pipeline instance created.
2022-04-25 17:58:25,481 - stpipe.Image2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.
2022-04-25 17:58:25,483 - stpipe.Image2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.
2022-04-25 17:58:25,485 - stpipe.Image2Pipeline.flat_field - INFO - FlatFieldStep instance created.
2022-04-25 17:58:25,486 - stpipe.Image2Pipeline.photom - INFO - PhotomStep instance created.
2022-04-25 17:58:25,489 - stpipe.Image2Pipeline.resample - INFO - ResampleStep instance created.
2022-04-25 17:58:25,495 - stpipe.Image2Pipeline - INFO - Image2Pipeline instance created.
2022-04-25 17:58:25,497 - stpipe.Image2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.
2022-04-25 17:58:25,499 - stpipe.Image2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.
2022-04-25 17:58:25,501 - stpipe.Image2Pipeline.flat_field - INFO - FlatFieldStep instance created.
2022-04-25 17:58:25,503 

In [19]:
image2 = Image2Pipeline()
image2.output_dir = path_stage2
image2.save_results = True
image2.run(path_stage2+'lvl2-asn.json')

2022-04-25 18:02:08,189 - stpipe.Image2Pipeline - INFO - Image2Pipeline instance created.
2022-04-25 18:02:08,191 - stpipe.Image2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.
2022-04-25 18:02:08,194 - stpipe.Image2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.
2022-04-25 18:02:08,196 - stpipe.Image2Pipeline.flat_field - INFO - FlatFieldStep instance created.
2022-04-25 18:02:08,199 - stpipe.Image2Pipeline.photom - INFO - PhotomStep instance created.
2022-04-25 18:02:08,201 - stpipe.Image2Pipeline.resample - INFO - ResampleStep instance created.


ERROR! Session/line number was not unique in

2022-04-25 18:02:08,342 - stpipe.Image2Pipeline - INFO - Step Image2Pipeline running with args ('/home/cperrot/Work/code/MIRISIM/corono/corono_S2/lvl2-asn.json',).


 database. History logging moved to new session

2022-04-25 18:02:08,349 - stpipe.Image2Pipeline - INFO - Step Image2Pipeline parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': '/home/cperrot/Work/code/MIRISIM/corono/corono_S2/', 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': True, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'save_bsub': False, 'steps': {'bkg_subtract': {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'save_combined_background': False, 'sigma': 3.0, 'maxiters': None}, 'assign_wcs': {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': 

 118


2022-04-25 18:02:09,094 - stpipe.Image2Pipeline - INFO - Prefetching reference files for dataset: 'det_image_seq1_MIRIMAGE_F1000Wexp1_rate.fits' reftypes = ['area', 'camera', 'collimator', 'dflat', 'disperser', 'distortion', 'drizpars', 'fflat', 'filteroffset', 'flat', 'fore', 'fpa', 'ifufore', 'ifupost', 'ifuslicer', 'msa', 'ote', 'photom', 'regions', 'sflat', 'specwcs', 'wavelengthrange', 'wfssbkg']
2022-04-25 18:02:10,252 - stpipe.Image2Pipeline - INFO - Prefetch for AREA reference file is '/home/cperrot/crds_cache/references/jwst/miri/jwst_miri_area_0004.fits'.
2022-04-25 18:02:10,253 - stpipe.Image2Pipeline - INFO - Prefetch for CAMERA reference file is 'N/A'.
2022-04-25 18:02:10,254 - stpipe.Image2Pipeline - INFO - Prefetch for COLLIMATOR reference file is 'N/A'.
2022-04-25 18:02:10,255 - stpipe.Image2Pipeline - INFO - Prefetch for DFLAT reference file is 'N/A'.
2022-04-25 18:02:10,255 - stpipe.Image2Pipeline - INFO - Prefetch for DISPERSER reference file is 'N/A'.
2022-04-25 18:

2022-04-25 18:02:11,594 - stpipe.Image2Pipeline.photom - INFO -  exp_type: MIR_IMAGE
2022-04-25 18:02:11,595 - stpipe.Image2Pipeline.photom - INFO -  filter: F1000W
2022-04-25 18:02:11,654 - stpipe.Image2Pipeline.photom - INFO - Pixel area map copied to output.
2022-04-25 18:02:11,657 - stpipe.Image2Pipeline.photom - INFO -  subarray: FULL
2022-04-25 18:02:11,659 - stpipe.Image2Pipeline.photom - INFO - PHOTMJSR value: 0.849523
2022-04-25 18:02:11,679 - stpipe.Image2Pipeline.photom - INFO - Step photom done
2022-04-25 18:02:11,798 - stpipe.Image2Pipeline.resample - INFO - Step resample running with args (<ImageModel(1024, 1032) from det_image_seq1_MIRIMAGE_F1000Wexp1_rate.fits>,).
2022-04-25 18:02:11,800 - stpipe.Image2Pipeline.resample - INFO - Step resample parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': True, 'skip': False, 'suffix': 'i2d', 'search_

2022-04-25 18:02:16,781 - stpipe.Image2Pipeline.photom - INFO - Step photom done
2022-04-25 18:02:16,917 - stpipe.Image2Pipeline.resample - INFO - Step resample running with args (<ImageModel(1024, 1032) from det_image_seq1_MIRIMAGE_F1000Wexp2_rate.fits>,).
2022-04-25 18:02:16,919 - stpipe.Image2Pipeline.resample - INFO - Step resample parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': True, 'skip': False, 'suffix': 'i2d', 'search_output_file': True, 'input_dir': '/home/cperrot/Work/code/MIRISIM/corono/corono_S2', 'pixfrac': 1.0, 'kernel': 'square', 'fillval': 'INDEF', 'weight_type': 'ivm', 'output_shape': None, 'crpix': None, 'crval': None, 'rotation': None, 'pixel_scale_ratio': 1.0, 'pixel_scale': None, 'single': False, 'blendheaders': False, 'allowed_memory': None}
2022-04-25 18:02:16,950 - stpipe.Image2Pipeline.resample - INFO - Drizpars reference fi

[<ImageModel(1024, 1032) from lvl2-asn_exposure1_cal.fits>,
 <ImageModel(1024, 1032) from lvl2-asn_exposure2_cal.fits>]

In [19]:
sstring = path_stage2+'det*cal.fits'
stage2_files = sorted(glob.glob(sstring))
print('Found ' + str(len(stage2_files)) + ' input files to process')

Found 2 input files to process


In [23]:
writel3asn(stage2_files, 'lvl3-asn.json','Level3')

Run stage 3 pipeline

Function of stage 3 corono

In [13]:
#definition des different DIR
output_dir_S1  = '/home/cperrot/Work/code/MIRISIM/corono/corono_S1/'
output_dir_raw = '/home/cperrot/Work/code/MIRISIM/corono/20220328_162102_mirisim/det_images/'

In [ ]:
#definition des param


In [14]:
# Download a MIRI parameter refrence file for the pipeline
miri_param_info = ('https://stsci.box.com/shared/static/cbsj8v6ull992n4jdkzlg5485arsmars.asdf',
                   'miri_detector1_param_file.asdf')
miri_param_ref_file = download_files(miri_param_info, output_dir_S1, force=False)[0]

miri_detector1_param_file.asdf already exists, skipping download...


In [24]:
raw_files = ['det_image_seq1_MIRIMAGE_F1000Wexp1.fits','det_image_seq1_MIRIMAGE_F1000Wexp2.fits']
for i in raw_files:
    call_output = calwebb_detector1.Detector1Pipeline.call(output_dir_raw+i,
                                                       output_dir=output_dir_S1,
                                                       save_results=True,
                                                       config_file=output_dir_S1+miri_param_ref_file)

2022-04-04 16:55:05,486 - stpipe.Detector1Pipeline - INFO - Detector1Pipeline instance created.
2022-04-04 16:55:05,487 - stpipe.Detector1Pipeline.group_scale - INFO - GroupScaleStep instance created.
2022-04-04 16:55:05,489 - stpipe.Detector1Pipeline.dq_init - INFO - DQInitStep instance created.
2022-04-04 16:55:05,492 - stpipe.Detector1Pipeline.saturation - INFO - SaturationStep instance created.
2022-04-04 16:55:05,493 - stpipe.Detector1Pipeline.ipc - INFO - IPCStep instance created.
2022-04-04 16:55:05,495 - stpipe.Detector1Pipeline.superbias - INFO - SuperBiasStep instance created.
2022-04-04 16:55:05,496 - stpipe.Detector1Pipeline.refpix - INFO - RefPixStep instance created.
2022-04-04 16:55:05,498 - stpipe.Detector1Pipeline.rscd - INFO - RscdStep instance created.
2022-04-04 16:55:05,500 - stpipe.Detector1Pipeline.firstframe - INFO - FirstFrameStep instance created.
2022-04-04 16:55:05,503 - stpipe.Detector1Pipeline.lastframe - INFO - LastFrameStep instance created.
2022-04-04 1

2022-04-04 16:55:05,734 - stpipe.Detector1Pipeline - INFO - Prefetch for REFPIX reference file is 'N/A'.
2022-04-04 16:55:05,734 - stpipe.Detector1Pipeline - INFO - Prefetch for RESET reference file is '/home/cperrot/crds_cache/references/jwst/miri/jwst_miri_reset_0047.fits'.
2022-04-04 16:55:05,735 - stpipe.Detector1Pipeline - INFO - Prefetch for RSCD reference file is '/home/cperrot/crds_cache/references/jwst/miri/jwst_miri_rscd_0014.fits'.
2022-04-04 16:55:05,735 - stpipe.Detector1Pipeline - INFO - Prefetch for SATURATION reference file is '/home/cperrot/crds_cache/references/jwst/miri/jwst_miri_saturation_0026.fits'.
2022-04-04 16:55:05,736 - stpipe.Detector1Pipeline - INFO - Prefetch for SUPERBIAS reference file is 'N/A'.
2022-04-04 16:55:05,736 - stpipe.Detector1Pipeline - INFO - Prefetch for TRAPDENSITY reference file is 'N/A'.
2022-04-04 16:55:05,737 - stpipe.Detector1Pipeline - INFO - Prefetch for TRAPPARS reference file is 'N/A'.
2022-04-04 16:55:05,737 - stpipe.Detector1Pipe

2022-04-04 16:55:08,632 - stpipe.Detector1Pipeline.rscd - INFO - Using RSCD reference file /home/cperrot/crds_cache/references/jwst/miri/jwst_miri_rscd_0014.fits
2022-04-04 16:55:08,725 - stpipe.Detector1Pipeline.rscd - WARNING - Too few groups to apply RSCD correction
2022-04-04 16:55:08,726 - stpipe.Detector1Pipeline.rscd - WARNING - RSCD step will be skipped
2022-04-04 16:55:08,732 - stpipe.Detector1Pipeline.rscd - INFO - Step rscd done
2022-04-04 16:55:08,843 - stpipe.Detector1Pipeline.dark_current - INFO - Step dark_current running with args (<RampModel(3, 5, 1024, 1032) from det_image_seq1_MIRIMAGE_F1000Wexp1.fits>,).
2022-04-04 16:55:08,845 - stpipe.Detector1Pipeline.dark_current - INFO - Step dark_current parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': '/home/cperrot/Work/code/MIRISIM/corono/corono_S1/', 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': True, 'suffix': None, 'search_

2022-04-04 16:55:14,953 - stpipe.Detector1Pipeline.ramp_fit - INFO - Step ramp_fit done
2022-04-04 16:55:15,061 - stpipe.Detector1Pipeline.gain_scale - INFO - Step gain_scale running with args (<ImageModel(1024, 1032) from det_image_seq1_MIRIMAGE_F1000Wexp1_jump.fits>,).
2022-04-04 16:55:15,063 - stpipe.Detector1Pipeline.gain_scale - INFO - Step gain_scale parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': 'gain_scale', 'search_output_file': True, 'input_dir': '/home/cperrot/Work/code/MIRISIM/corono/20220328_162102_mirisim/det_images'}
2022-04-04 16:55:15,109 - stpipe.Detector1Pipeline.gain_scale - INFO - GAINFACT not found in gain reference file
2022-04-04 16:55:15,110 - stpipe.Detector1Pipeline.gain_scale - INFO - Step will be skipped
2022-04-04 16:55:15,115 - stpipe.Detector1Pipeline.gain_scale - INFO - Step gain_scale 

2022-04-04 16:55:15,939 - stpipe.Detector1Pipeline - INFO - Prefetching reference files for dataset: 'det_image_seq1_MIRIMAGE_F1000Wexp2.fits' reftypes = ['gain', 'linearity', 'mask', 'persat', 'readnoise', 'refpix', 'reset', 'rscd', 'saturation', 'superbias', 'trapdensity', 'trappars']
2022-04-04 16:55:15,944 - stpipe.Detector1Pipeline - INFO - Prefetch for GAIN reference file is '/home/cperrot/crds_cache/references/jwst/miri/jwst_miri_gain_0008.fits'.
2022-04-04 16:55:15,945 - stpipe.Detector1Pipeline - INFO - Prefetch for LINEARITY reference file is '/home/cperrot/crds_cache/references/jwst/miri/jwst_miri_linearity_0024.fits'.
2022-04-04 16:55:15,946 - stpipe.Detector1Pipeline - INFO - Prefetch for MASK reference file is '/home/cperrot/crds_cache/references/jwst/miri/jwst_miri_mask_0028.fits'.
2022-04-04 16:55:15,946 - stpipe.Detector1Pipeline - INFO - Prefetch for PERSAT reference file is 'N/A'.
2022-04-04 16:55:15,947 - stpipe.Detector1Pipeline - INFO - Prefetch for READNOISE refe

2022-04-04 16:55:17,773 - stpipe.Detector1Pipeline.linearity - INFO - Using Linearity reference file /home/cperrot/crds_cache/references/jwst/miri/jwst_miri_linearity_0024.fits
2022-04-04 16:55:18,056 - stpipe.Detector1Pipeline.linearity - INFO - Step linearity done
2022-04-04 16:55:18,164 - stpipe.Detector1Pipeline.rscd - INFO - Step rscd running with args (<RampModel(3, 5, 1024, 1032) from det_image_seq1_MIRIMAGE_F1000Wexp2.fits>,).
2022-04-04 16:55:18,166 - stpipe.Detector1Pipeline.rscd - INFO - Step rscd parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '/home/cperrot/Work/code/MIRISIM/corono/20220328_162102_mirisim/det_images', 'type': 'baseline'}
2022-04-04 16:55:18,189 - stpipe.Detector1Pipeline.rscd - INFO - Using RSCD reference file /home/cperrot/crds_cache/reference

2022-04-04 16:55:21,146 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using GAIN reference file: /home/cperrot/crds_cache/references/jwst/miri/jwst_miri_gain_0008.fits
2022-04-04 16:55:21,199 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using algorithm = ols
2022-04-04 16:55:21,199 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using weighting = optimal
2022-04-04 16:55:21,217 - stpipe.Detector1Pipeline.ramp_fit - INFO - Number of leading groups that are flagged as DO_NOT_USE: 1
2022-04-04 16:55:21,219 - stpipe.Detector1Pipeline.ramp_fit - INFO - MIRI dataset has all pixels in the final group flagged as DO_NOT_USE.
2022-04-04 16:55:25,605 - stpipe.Detector1Pipeline.ramp_fit - INFO - Number of groups per integration: 3
2022-04-04 16:55:25,607 - stpipe.Detector1Pipeline.ramp_fit - INFO - Number of integrations: 3
2022-04-04 16:55:25,686 - stpipe.Detector1Pipeline.ramp_fit - INFO - Step ramp_fit done
2022-04-04 16:55:25,800 - stpipe.Detector1Pipeline.gain_scale - INFO - Step gain_scale runni